In [1]:
import os
os.chdir('../')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle
import os
import sys
import numpy as np
import scipy.io as spio
import scipy as sp
from PIL import Image
from scipy.stats import pearsonr,binom,linregress
import numpy as np

In [2]:
def pairwise_corr_all(ground_truth, predictions):
    r = np.corrcoef(ground_truth, predictions)#cosine_similarity(ground_truth, predictions)#
    r = r[:len(ground_truth), len(ground_truth):]  # rows: groundtruth, columns: predicitons
    #print(r.shape)
    # congruent pairs are on diagonal
    congruents = np.diag(r)
    #print(congruents)
    
    # for each column (predicition) we should count the number of rows (groundtruth) that the value is lower than the congruent (e.g. success).
    success = r < congruents
    success_cnt = np.sum(success, 0)
    
    # note: diagonal of 'success' is always zero so we can discard it. That's why we divide by len-1
    perf = np.mean(success_cnt) / (len(ground_truth)-1)
    p = 1 - binom.cdf(perf*len(ground_truth)*(len(ground_truth)-1), len(ground_truth)*(len(ground_truth)-1), 0.5)
    
    return perf, p

def pairwise_corr_individuals(ground_truth, predictions):
    r = np.corrcoef(ground_truth, predictions)#cosine_similarity(ground_truth, predictions)#
    r = r[:len(ground_truth), len(ground_truth):]  # rows: groundtruth, columns: predicitons
    #print(r.shape)
    # congruent pairs are on diagonal
    congruents = np.diag(r)
    #print(congruents)
    
    # for each column (predicition) we should count the number of rows (groundtruth) that the value is lower than the congruent (e.g. success).
    success = r < congruents
    success_cnt = np.sum(success, 0)
    
    # note: diagonal of 'success' is always zero so we can discard it. That's why we divide by len-1
    perf = success_cnt / (len(ground_truth)-1)
    # p = 1 - binom.cdf(perf*len(ground_truth)*(len(ground_truth)-1), len(ground_truth)*(len(ground_truth)-1), 0.5)
    
    return perf


net_list = [
    ('inceptionv3','avgpool'),
    ('clip','final'),
    ('alexnet',2),
    ('alexnet',5),
    ('efficientnet','avgpool'),
    ('swav','avgpool')
    ]

In [3]:
num_test = 200
test_dir = 'cache/thingseeg2_preproc/eval_features/test_images'
feats_dir = 'cache/thingseeg2_preproc/eval_features/subj1_preproc_800ms'
distance_fn = sp.spatial.distance.correlation
pairwise_corrs = []
for (net_name,layer) in net_list:
    file_name = '{}/{}_{}.npy'.format(test_dir,net_name,layer)
    gt_feat = np.load(file_name)
    
    file_name = '{}/{}_{}.npy'.format(feats_dir,net_name,layer)
    eval_feat = np.load(file_name)
    
    gt_feat = gt_feat.reshape((len(gt_feat),-1))
    eval_feat = eval_feat.reshape((len(eval_feat),-1))

    print(net_name,layer)
    if net_name in ['efficientnet','swav']:
        print('distance: ',np.array([distance_fn(gt_feat[i],eval_feat[i]) for i in range(num_test)]).mean())
    else:
        # pairwise_corrs.append(pairwise_corr_all(gt_feat[:num_test],eval_feat[:num_test])[0])
        pairwise_corrs.append(pairwise_corr_individuals(gt_feat[:num_test],eval_feat[:num_test]))
        # print('pairwise corr: ',pairwise_corrs[-1])

inceptionv3 avgpool
clip final
alexnet 2
alexnet 5
efficientnet avgpool
distance:  0.9121800053943298
swav avgpool
distance:  0.5758596473932266


In [4]:
inception_inds = np.argsort(pairwise_corrs[0])[::-1]
clip_inds = np.argsort(pairwise_corrs[1])[::-1]
alexnet2_inds = np.argsort(pairwise_corrs[2])[::-1]
alexnet5_inds = np.argsort(pairwise_corrs[3])[::-1]

In [5]:
# train_path = 'data/things-eeg2_preproc/sub01/train_thingseeg2_avg_800ms.npy'
train_path = 'data/things-eeg2_preproc/train_thingseeg2_avg_800ms.npy'
train_eeg = np.load(train_path, mmap_mode='r')
# train_eeg_flattened = train_eeg.reshape(train_eeg.shape[0], -1)
# test_path = 'data/things-eeg2_preproc/sub01/test_thingseeg2_avg_800ms.npy'
test_path = 'data/things-eeg2_preproc/test_thingseeg2_avg_800ms.npy'
test_eeg = np.load(test_path, mmap_mode='r')
test_eeg = test_eeg[clip_inds]
# test_eeg_flattened = test_eeg.reshape(test_eeg.shape[0], -1)
print(train_eeg.shape, test_eeg.shape)

(16540, 17, 80) (200, 17, 80)


In [6]:
test_eeg_0 = test_eeg[0]
test_eeg_4 = test_eeg[4]

In [7]:
time_window = 5
stride = 1
n_examples = 80 - time_window + 1
synthetic_eeg_0to4 = np.zeros((n_examples, 17, 80))
synthetic_eeg_4to0 = np.zeros((n_examples, 17, 80))

In [8]:
for i_example in range(n_examples - 1):
    synthetic_eeg_0to4[i_example] = test_eeg_4
    synthetic_eeg_4to0[i_example] = test_eeg_0
    start = i_example * stride
    end = start + time_window
    synthetic_eeg_0to4[i_example,:, start:end] = test_eeg_0[:, start:end]
    synthetic_eeg_4to0[i_example,:, start:end] = test_eeg_4[:, start:end]
synthetic_eeg_0to4[-1] = test_eeg_4
synthetic_eeg_4to0[-1] = test_eeg_0

In [9]:
dir = 'cache/thingseeg2_synthetic/'
if not os.path.exists(dir):
   os.makedirs(dir)
np.save(dir + 'sub01_0to4__5_1__800ms.npy', synthetic_eeg_0to4)
np.save(dir + 'sub01_4to0__5_1__800ms.npy', synthetic_eeg_4to0)